In [ ]:
import pandas as pd
import re
import string

df=pd.read_json("./data/arxiv-metadata-oai-snapshot.json",lines=True) #download from https://www.kaggle.com/datasets/Cornell-University/arxiv

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df=df[["abstract","update_date"]].rename(columns={"abstract":"content","year_update":"time"})
df['time'] = pd.to_datetime(df['time'], infer_datetime_format=True)
df['time'] = df['time'].dt.to_period('Y')
df=df.sort_values(by="time")

df['content'] = df['content'].str.replace(r'@\w+', '')
df['content'] = df['content'].str.replace('\n', ' ').replace('\r', '')
df['content'] = df['content'].apply(lambda x: remove_punct(x))

df=df.dropna()

df=df.sort_values(by="time")
df=df.reset_index(drop=True)
df=df.reset_index()

In [ ]:
from antm import ANTM

window_size=3
overlap=1

#take a random sample for example
dt=df.sample(n = 1000)
dt=dt.sort_values("time")
dt=dt.reset_index(drop=True)
dt=dt.reset_index()

model=ANTM(dt,overlap,window_size,mode="data2vec",num_words=10,path="./saved_data")

#learn the model and save it
model.fit(save=True)